## Load

In [1]:
import os
from collections import defaultdict
import math
import numpy as np 
import random
import re
import torch
import torch.nn as nn
from itertools import cycle
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
from torch.optim.lr_scheduler import MultiStepLR
from tqdm.auto import tqdm

# Used to get the data
from sklearn.metrics import ndcg_score

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import nltk
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.collocations import BigramAssocMeasures, BigramCollocationFinder
nltk.download('stopwords')

import matplotlib.pyplot as plt 
import matplotlib
matplotlib.use('Agg')

seed = 33
import pandas as pd

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/chrisliu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Preprocess config

In [2]:
config = {}

config["dataset"] = "CNN" # "IMDB" "CNN", "PubMed"
config["n_document"] = 10000000
config["normalize_word_embedding"] = True
config["min_word_freq_threshold"] = 20
config["topk_word_freq_threshold"] = 100
config["document_vector_agg_weight"] = 'IDF' # ['mean', 'IDF', 'uniform', 'gaussian', 'exponential', 'pmi']
config["document_vector_weight_normalize"] = False # weighted sum or mean, True for sum, False for mean 
config["select_topk_TFIDF"] = None # ignore
config["embedding_file"] = "../data/glove.6B.100d.txt"
config["topk"] = [10, 30, 50]


In [3]:
def in_notebook():
    try:
        from IPython import get_ipython
        if 'IPKernelApp' not in get_ipython().config:  # pragma: no cover
            return False
    except ImportError:
        return False
    return True

In [4]:
def load_word2emb(embedding_file):
    word2embedding = dict()
    word_dim = int(re.findall(r".(\d+)d", embedding_file)[0])

    with open(embedding_file, "r") as f:
        for line in tqdm(f):
            line = line.strip().split()
            word = line[0]
            embedding = list(map(float, line[1:]))
            word2embedding[word] = np.array(embedding)

    print("Number of words:%d" % len(word2embedding))

    return word2embedding

word2embedding = load_word2emb(config["embedding_file"])


Number of words:400000


In [5]:
def normalize_wordemb(word2embedding):
    # Every word emb should have norm 1
    
    word_emb = []
    word_list = []
    for word, emb in word2embedding.items():
        word_list.append(word)
        word_emb.append(emb)

    word_emb = np.array(word_emb)

    for i in range(len(word_emb)):
        norm = np.linalg.norm(word_emb[i])
        word_emb[i] = word_emb[i] / norm

    for word, emb in tqdm(zip(word_list, word_emb)):
        word2embedding[word] = emb
    return word2embedding

if config["normalize_word_embedding"]:
    normalize_wordemb(word2embedding)

In [6]:
class Vocabulary:
    def __init__(self, word2embedding, config):
        # The low frequency words will be assigned as <UNK> token
        self.itos = {0: "<UNK>"}
        self.stoi = {"<UNK>": 0}
        
        self.word2embedding = word2embedding
        self.config = config

        self.word_freq_in_corpus = defaultdict(int)
        self.IDF = {}
        self.ps = PorterStemmer()
        self.stop_words = set(stopwords.words('english'))
        
        self.word_dim = len(word2embedding['the'])
    def __len__(self):
        return len(self.itos)

    def tokenizer_eng(self, text):
        text = re.sub(r'[^A-Za-z0-9 ]+', '', text)
        text = text.strip().split()
        
        return [self.ps.stem(w) for w in text if w.lower() not in self.stop_words]
    
    def read_raw(self):        
        if self.config["dataset"] == 'IMDB':
            data_file_path = '../data/IMDB.txt'
        elif self.config["dataset"] == 'CNN':
            data_file_path = '../data/CNN.txt'
        elif self.config["dataset"] == 'PubMed':
            data_file_path = '../data/PubMed.txt'
        
        # raw documents
        self.raw_documents = []
        with open(data_file_path,'r',encoding='utf-8') as f:
            for line in tqdm(f, desc="Loading documents"):
                self.raw_documents.append(line.strip("\n"))
                
        return self.raw_documents
    
    def build_vocabulary(self):
        sentence_list = self.raw_documents
        
        self.doc_freq = defaultdict(int) # # of document a word appear
        self.document_num = len(sentence_list)
        self.word_vectors = [[0]*self.word_dim] # unknown word emb
        
        for sentence in tqdm(sentence_list, desc="Preprocessing documents"):
            # for doc_freq
            document_words = set()
            
            for word in self.tokenizer_eng(sentence):
                # pass unknown word
                if word not in self.word2embedding:
                    continue
                    
                # calculate word freq
                self.word_freq_in_corpus[word] += 1
                document_words.add(word)
                
            for word in document_words:
                self.doc_freq[word] += 1
        
        # calculate IDF
        print('doc num', self.document_num)
        for word, freq in self.doc_freq.items():
            self.IDF[word] = math.log(self.document_num / (freq+1))
        
        # delete less freq words:
        delete_words = []
        for word, v in self.word_freq_in_corpus.items():
            if v < self.config["min_word_freq_threshold"]:
                delete_words.append(word)     
        for word in delete_words:
            del self.IDF[word]    
            del self.word_freq_in_corpus[word]    
        
        # delete too freq words
        print('eliminate freq words')
        IDF = [(word, freq) for word, freq in self.IDF.items()]
        IDF.sort(key=lambda x: x[1])

        for i in range(self.config["topk_word_freq_threshold"]):
            print(word)
            word = IDF[i][0]
            del self.IDF[word]
            del self.word_freq_in_corpus[word]
        
        # construct word_vectors
        idx = 1
        for word in self.word_freq_in_corpus:
            self.word_vectors.append(self.word2embedding[word])
            self.stoi[word] = idx
            self.itos[idx] = word
            idx += 1
            
    def init_word_weight(self,sentence_list, agg):
        if agg == 'mean':
            self.word_weight = {word: 1 for word in self.IDF.keys()}
        elif agg == 'IDF':
            self.word_weight = self.IDF
        elif agg == 'uniform':
            self.word_weight = {word: np.random.uniform(low=0.0, high=1.0) for word in self.IDF.keys()}
        elif agg == 'gaussian':
            mu, sigma = 10, 1 # mean and standard deviation
            self.word_weight = {word: np.random.normal(mu, sigma) for word in self.IDF.keys()}
        elif agg == 'exponential':
            self.word_weight = {word: np.random.exponential(scale=1.0) for word in self.IDF.keys()}
        elif agg == 'pmi':
            trigram_measures = BigramAssocMeasures()
            self.word_weight = defaultdict(int)
            corpus = []

            for text in tqdm(sentence_list):
                corpus.extend(text.split())

            finder = BigramCollocationFinder.from_words(corpus)
            for pmi_score in finder.score_ngrams(trigram_measures.pmi):
                pair, score = pmi_score
                self.word_weight[pair[0]] += score
                self.word_weight[pair[1]] += score
                
    def calculate_document_vector(self):
        # Return
        # document_vectors: weighted sum of word emb
        # document_answers_idx: doc to word index list
        # document_answers_wsum: word weight summation, e.g. total TFIDF score of a doc
        
        document_vectors = [] 
        document_answers = []
        document_answers_wsum = []
        
        sentence_list = self.raw_documents
        agg = self.config["document_vector_agg_weight"]
        n_document = self.config["n_document"]
        select_topk_TFIDF = self.config["select_topk_TFIDF"]
        
        self.init_word_weight(sentence_list, agg)
        for sentence in tqdm(sentence_list[:min(n_document, len(sentence_list))], desc="calculate document vectors"):
            document_vector = np.zeros(len(self.word_vectors[0]))
            select_words = []
            for word in self.tokenizer_eng(sentence):
                # pass unknown word
                if word not in self.stoi:
                    continue
                else:
                    select_words.append(word)

            # select topk TDIDF
            if select_topk_TFIDF is not None:
                doc_TFIDF = defaultdict(float)
                for word in select_words:    
                    doc_TFIDF[word] += self.IDF[word]

                doc_TFIDF_l = [(word, TFIDF) for word, TFIDF in doc_TFIDF.items()]
                doc_TFIDF_l.sort(key=lambda x:x[1], reverse=True)
                
                select_topk_words = set(list(map(lambda x:x[0], doc_TFIDF_l[:select_topk_TFIDF])))
                select_words = [word for word in select_words if word in select_topk_words]
            else:
                pass
            
            total_weight = 0
            # aggregate to doc vectors
            for word in select_words:
                document_vector += np.array(self.word2embedding[word]) * self.word_weight[word]
                total_weight += self.word_weight[word]
                
            if len(select_words) == 0:
                print('error', sentence)
                continue
            else:
                if self.config["document_vector_weight_normalize"]:
                    document_vector /= total_weight
            
            document_vectors.append(document_vector)
            document_answers.append(select_words)
            document_answers_wsum.append(total_weight)
        
        # get answers
        document_answers_idx = []    
        for ans in document_answers:
            ans_idx = []
            for token in ans:
                if token in self.stoi:
                    ans_idx.append(self.stoi[token])                    
            document_answers_idx.append(ans_idx)
        
        self.document_vectors = document_vectors
        self.document_answers_idx = document_answers_idx
        self.document_answers_wsum = document_answers_wsum
        
        return document_vectors, document_answers_idx, document_answers_wsum
        
    def numericalize(self, text):
        tokenized_text = self.tokenizer_eng(text)

        return [
            self.stoi[token] if token in self.stoi else self.stoi["<UNK>"]
            for token in tokenized_text
        ]
    
    def check_docemb(self):
        word_vectors = np.array(self.word_vectors)
        pred = np.zeros(word_vectors.shape[1])
        cnt = 0

        for word_idx in self.document_answers_idx[0]:
            pred += word_vectors[word_idx] * self.word_weight[self.itos[word_idx]]
            cnt += self.word_weight[self.itos[word_idx]]
        
        if self.config["document_vector_weight_normalize"]:
            pred /= cnt
        assert np.sum(self.document_vectors[0]) - np.sum(pred) == 0

In [7]:
def build_vocab(config, word2embedding):
    # build vocabulary
    vocab = Vocabulary(word2embedding, config)
    vocab.read_raw()
    vocab.build_vocabulary()
    vocab_size = len(vocab)
    # get doc emb
    vocab.calculate_document_vector()
    vocab.check_docemb()
    
    return vocab

vocab = build_vocab(config, word2embedding)


doc num 19026
eliminate freq words
paroxysm
subject
line
organ
write
univers
one
would
use
like
get
know
dont
think
time
make
also
say
go
im
could
want
new
work
good
well
way
need
look
even
anyon
thing
see
tri
thank
much
year
world
system
right
problem
may
take
mani
two
first
seem
question
pleas
1
state
us
come
2
post
help
call
usa
point
sinc
find
read
still
back
mean
ive
give
email
sure
differ
might
run
cant
reason
last
day
interest
case
let
person
said
never
start
doesnt
tell
better
ask
got
without
follow
part
lot
3
number
put
fact
gener
inform
actual
that


error |> 
error 
error 
error Mikael Fredriksson
error 
error -------------------------------------------------
error email: mikael_fredriksson@macexchange.se
error 
error FIDO 2:203/211
error  
error  
error   
error  
error  
error  
error  
error  
error  
error  
error  
error  
error  
error  
error  
error  
error  
error  
error  
error  
error  
error  
error  
error  
error  
error  
error  
error    
error  
error  
error  
error  
error  
error  
error  
error  
error  
error  
error  
error  
error  
error  
error  
error  
error  
error  
error  
error  
error  
error  
error  
error  
error  
error  
error  
error  
error  
error  
error  
error  
error    
error  
error  
error  
error  
error  
error  
error  
error  



In [8]:
print("Finish building dataset!")
print(f"Number of documents:{len(vocab.raw_documents)}")
print(f"Number of words:{len(vocab)}")

l = list(map(len, vocab.document_answers_idx))
print("Average length of document:", np.mean(l))

Finish building dataset!
Number of documents:19026
Number of words:7602
Average length of document: 84.86410175216382


In [9]:
word_vectors = np.array(vocab.word_vectors)
print(word_vectors.shape)

document_vectors = np.array(vocab.document_vectors)
print(document_vectors.shape)

document_answers_wsum = np.array(vocab.document_answers_wsum).reshape(-1, 1)
print(document_answers_wsum.shape)

# create weight_ans
document_answers_idx = vocab.document_answers_idx

# random shuffle
shuffle_idx = list(range(len(document_vectors)))
random.Random(seed).shuffle(shuffle_idx)

document_vectors = document_vectors[shuffle_idx]
document_answers_wsum = document_answers_wsum[shuffle_idx]
document_answers_idx = [document_answers_idx[idx] for idx in shuffle_idx]

(7602, 100)
(18948, 100)
(18948, 1)


In [10]:
# onthot_ans: word freq matrix
# weight_ans: TFIDF matrix

onehot_ans = np.zeros((len(document_answers_idx), word_vectors.shape[0]))
weight_ans = np.zeros((len(document_answers_idx), word_vectors.shape[0]))
print(weight_ans.shape)

for i in tqdm(range(len(document_answers_idx))):
    for word_idx in document_answers_idx[i]:
        weight_ans[i, word_idx] += vocab.word_weight[vocab.itos[word_idx]]
        onehot_ans[i, word_idx] += 1

(18948, 7602)


## Results

In [11]:
final_results = []
select_columns = ['model']
for topk in config["topk"]:
    select_columns.append('F1@{}'.format(topk))
for topk in config["topk"]:
    select_columns.append('ndcg@{}'.format(topk))
select_columns.append('ndcg@all')
select_columns

['model',
 'F1@10',
 'F1@30',
 'F1@50',
 'ndcg@10',
 'ndcg@30',
 'ndcg@50',
 'ndcg@all']

## setting training size

In [12]:
train_size_ratio = 1
train_size = int(len(document_answers_idx) * train_size_ratio)
train_size

18948

## Top K freq word

In [13]:
topk_results = {}

In [14]:
test_ans = document_answers_idx[:train_size]

In [15]:
word_freq = [(word, freq) for word, freq in vocab.word_freq_in_corpus.items()]
word_freq.sort(key=lambda x:x[1], reverse=True)
word_freq[:10]

[('x', 6539),
 ('god', 5208),
 ('file', 4918),
 ('0', 4520),
 ('window', 4444),
 ('program', 4201),
 ('drive', 3633),
 ('4', 3528),
 ('game', 3474),
 ('govern', 3268)]

In [16]:
def topk_word_evaluation(k=50):
    topk_word = [word for (word, freq) in word_freq[:k]]

    pr, re = [], []
    for ans in tqdm(test_ans):
        ans = set(ans)
        ans = [vocab.itos[a] for a in ans]

        hit = []
        for word in ans:
            if word in topk_word:
                hit.append(word)

        precision = len(hit) / k
        recall = len(hit) / len(ans)
        pr.append(precision)
        re.append(recall)

    pr = np.mean(pr)
    re = np.mean(re)
    f1 = 2 * pr * re / (pr + re) if (pr + re) != 0 else 0
    print('top {} word'.format(k))
    print('percision', np.mean(pr))
    print('recall', np.mean(re))
    print('F1', f1)
    return f1


for topk in config['topk']:
    topk_results["F1@{}".format(topk)] = topk_word_evaluation(k=topk)



top 10 word
percision 0.06950073886426009
recall 0.016353539716943234
F1 0.026477028568787433



top 30 word
percision 0.07639680529167546
recall 0.04850870113830837
F1 0.059339414277828975



top 50 word
percision 0.07682921680388433
recall 0.07978013464427035
F1 0.07827687433156728


In [17]:
def topk_word_evaluation_NDCG(k=50):
    freq_word =[word for (word, freq) in word_freq]
    freq_word_idx = [vocab.stoi[word] for word in freq_word if word in vocab.stoi]
    
    scores = np.zeros(len(vocab.word_vectors))
    for rank, idx in enumerate(freq_word_idx):
        scores[idx] = len(vocab.word_vectors) - rank
    
    NDCGs = []
    
    for ans in tqdm(test_ans):
        weight_ans = np.zeros(len(vocab.word_vectors))
        
        for word_idx in ans:
            if word_idx == 0:
                continue
            word = vocab.itos[word_idx]
            weight_ans[word_idx] += vocab.IDF[word]

        NDCG_score = ndcg_score(weight_ans.reshape(1,-1), scores.reshape(1,-1), k=k)
        NDCGs.append(NDCG_score)

    print('top {} NDCG:{}'.format(k, np.mean(NDCGs)))
    
    return np.mean(NDCGs)


for topk in config['topk']:
    topk_results["ndcg@{}".format(topk)] = topk_word_evaluation_NDCG(k=topk)
    
topk_results["ndcg@all"] = topk_word_evaluation_NDCG(k=None)



top 10 NDCG:0.028211750096691315



top 30 NDCG:0.03761860810723896



top 50 NDCG:0.04587430885683149



top None NDCG:0.31437606429732357


In [18]:
topk_results["model"] = "topk"
final_results.append(pd.Series(topk_results))

## MLP Decoder

In [19]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [20]:
class MLPDecoderDataset(Dataset):
    def __init__(self, 
                 doc_vectors,
                 weight_ans,
                 topk=50):
        self.doc_vectors = torch.FloatTensor(doc_vectors)
        self.weight_ans = torch.FloatTensor(weight_ans)
        self.weight_ans_s = torch.argsort(self.weight_ans, dim=1, descending=True)
        self.weight_ans_s[:, topk:] = -1
        
        assert len(doc_vectors) == len(weight_ans)
        
    def __getitem__(self, idx):
        return self.doc_vectors[idx], self.weight_ans[idx], self.weight_ans_s[idx]

    def __len__(self):
        return len(self.doc_vectors)

In [21]:
batch_size = 100

train_size_ratio = 0.9
train_size = int(len(document_answers_idx) * train_size_ratio)
train_size

print('train size', train_size)
print('valid size', len(document_vectors) - train_size)

train_dataset = MLPDecoderDataset(document_vectors[:train_size], weight_ans[:train_size], topk=50)
train_loader  = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

valid_dataset = MLPDecoderDataset(document_vectors[train_size:], weight_ans[train_size:], topk=50)
valid_loader  = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=True, pin_memory=True)

train size 17053
valid size 1895


In [22]:
class MLPDecoder(nn.Module):
    def __init__(self, doc_emb_dim, num_words, h_dim=300):
        super().__init__()
        
        self.fc1 = nn.Linear(doc_emb_dim, h_dim) 
#         self.fc2 = nn.Linear(h_dim, h_dim)
#         self.fc3 = nn.Linear(h_dim, h_dim)
        self.fc4 = nn.Linear(h_dim, num_words)
        self.dropout = nn.Dropout(p=0.2)
        
    def forward(self, x):
        x = F.tanh(self.fc1(x))
#         x = F.tanh(self.fc2(x))
#         x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)
        
        return x

In [23]:
def _dcg(target):
    """Computes Discounted Cumulative Gain for input tensor."""
    denom = torch.log2(torch.arange(target.shape[-1], device=target.device) + 2.0)
    return (target / denom).sum(dim=-1)

def retrieval_normalized_dcg_all(preds, target, k=None):
    """Computes `Normalized Discounted Cumulative Gain`_ (for information retrieval).
    ``preds`` and ``target`` should be of the same shape and live on the same device.
    ``target`` must be either `bool` or `integers` and ``preds`` must be `float`,
    otherwise an error is raised.
    Args:
        preds: estimated probabilities of each document to be relevant.
        target: ground truth about each document relevance.
        k: consider only the top k elements (default: None, which considers them all)
    Return:
        a single-value tensor with the nDCG of the predictions ``preds`` w.r.t. the labels ``target``.
    Raises:
        ValueError:
            If ``k`` parameter is not `None` or an integer larger than 0
    Example:
        >>> from torchmetrics.functional import retrieval_normalized_dcg
        >>> preds = torch.tensor([.1, .2, .3, 4, 70])
        >>> target = torch.tensor([10, 0, 0, 1, 5])
        >>> retrieval_normalized_dcg(preds, target)
        tensor(0.6957)
    """
    k = [preds.shape[-1]] if k is None else k + [preds.shape[-1]]
    
    assert preds.shape == target.shape and max(k) <= preds.shape[-1]
    
    if not isinstance(k, list):
        raise ValueError("`k` has to be a list of positive integer or None")

    sorted_target = target.gather(1, torch.argsort(preds, dim=-1, descending=True))
    ideal_target = torch.sort(target, descending=True)[0]
    
    ndcg_scores = {}
    for topk in k:
        sorted_target_k = sorted_target[:,:topk]
        ideal_target_k = ideal_target[:,:topk]
        
        ideal_dcg_k = _dcg(ideal_target_k)
        target_dcg_k = _dcg(sorted_target_k)
        
        # filter undefined scores
        target_dcg_k /= ideal_dcg_k
        
        if topk == preds.shape[-1]:
            topk = 'all'
        ndcg_scores[topk] = target_dcg_k.mean().item()
        
    return ndcg_scores

def retrieval_precision_all(preds, target, k = [10]):
        
    assert preds.shape == target.shape and max(k) <= preds.shape[-1]
    
    if not isinstance(k, list):
        raise ValueError("`k` has to be a list of positive integer")
        
    precision_scores = {}
    target_onehot = target > 0
    
    for topk in k:
        relevant = target_onehot.gather(1, preds.topk(topk, dim=-1)[1])
        relevant = relevant.sum(axis=1).float()
        relevant /= topk    
        precision_scores[topk] = relevant.mean().item()
    
    return precision_scores

In [24]:
import timeit
from torchmetrics.functional import retrieval_normalized_dcg, retrieval_precision
from sklearn.metrics import ndcg_score

def evaluate_MLPDecoder(model, data_loader):
    results = {}
    model.eval()
    
    pred_all = []
    target_all = []
    
    # predict all data
    start = timeit.default_timer()
    for data in data_loader:
        doc_embs, target, _ = data
        
        doc_embs = doc_embs.to(device)
        target = target.to(device)
                
        pred = model(doc_embs)
        pred_all.append(pred)
        target_all.append(target)
        
    pred_all = torch.cat(pred_all, dim=0)
    target_all = torch.cat(target_all, dim=0)
    stop1 = timeit.default_timer()
    print('Time1: ', stop1 - start)  
    
    # Precision
    precision_scores = retrieval_precision_all(pred_all, target_all, k=config["topk"])
    for k, v in precision_scores.items():
        results['precision@{}'.format(k)] = v
    
    stop2 = timeit.default_timer()
    print('Time2: ', stop2 - stop1)  
    
    # NDCG
    ndcg_scores = retrieval_normalized_dcg_all(pred_all, target_all, k=config["topk"])
    for k, v in ndcg_scores.items():
        results['ndcg@{}'.format(k)] = v
        
    stop3 = timeit.default_timer()
    print('Time3: ', stop3 - stop2)      
    
    return results

In [25]:
# setting
lr = 0.05
momentum = 0.
weight_decay = 0
nesterov = False # True

n_epoch = 5000
verbose = True
valid_epoch = 50

h_dim = 3000

model = MLPDecoder(doc_emb_dim=document_vectors.shape[1], num_words=len(word_vectors), h_dim=h_dim).to(device)
model.train()

word_vectors_tensor = torch.FloatTensor(word_vectors).to(device)

opt = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay, nesterov=nesterov)
# opt = torch.optim.Adam(model.parameters(), lr=lr)
# criterion = nn.MSELoss(reduction='mean')
criterion = nn.MultiLabelMarginLoss(reduction='mean')
# criterion = nn.MultiLabelSoftMarginLoss(reduction='mean')

results = []
step = 0
clip_value = 1

for epoch in tqdm(range(n_epoch)):    
    train_loss_his = []
    valid_loss_his = []
    
    model.train()

    for data in train_loader:
        doc_embs, target, target_rank = data
        
        doc_embs = doc_embs.to(device)
        target = target.to(device)
        target_rank = target_rank.to(device)
        
        # MSE loss
        pred = model(doc_embs)     
#         loss = criterion(pred, target)
        loss = criterion(pred, target_rank)
#         loss = criterion(pred, torch.sign(target))
        
        # Model backwarding
        model.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_value_(model.parameters(), clip_value)
        
        opt.step()

        train_loss_his.append(loss.item())
        
    model.eval()
    for data in valid_loader:
        doc_embs, target, target_rank = data
        
        doc_embs = doc_embs.to(device)
        target = target.to(device)
        target_rank = target_rank.to(device)
        
        # MSE loss
        pred = model(doc_embs)     
        
#         loss = criterion(pred, target)
        loss = criterion(pred, target_rank)
#         loss = criterion(pred, torch.sign(target))
        
        valid_loss_his.append(loss.item())
    
    print("Epoch", epoch, np.mean(train_loss_his), np.mean(valid_loss_his))
    
    if epoch % valid_epoch == 0:
        res = {}
        res['epoch'] = epoch
        
        train_res_ndcg = evaluate_MLPDecoder(model, train_loader)
        valid_res_ndcg = evaluate_MLPDecoder(model, valid_loader)
        
        res.update(valid_res_ndcg)
        results.append(res)
        
        if verbose:
            print()
            print('train', train_res_ndcg)
            print('valid', valid_res_ndcg)
            


/home/chrisliu/virtual_env/py37/lib/python3.7/site-packages/torch/nn/functional.py:1794: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


Epoch 0 13.793183956927026 9.811852103785464
Time1:  0.5027011595666409
Time2:  0.05020587891340256
Time3:  0.16641565039753914
Time1:  0.0545343067497015
Time2:  0.00821242667734623
Time3:  0.018884481862187386

train {'precision@10': 0.12872809171676636, 'precision@30': 0.1125139370560646, 'precision@50': 0.11361167579889297, 'ndcg@10': 0.0913453921675682, 'ndcg@30': 0.10261578112840652, 'ndcg@50': 0.12063910067081451, 'ndcg@all': 0.3932613730430603}
valid {'precision@10': 0.12844327092170715, 'precision@30': 0.11134565621614456, 'precision@50': 0.1125699058175087, 'ndcg@10': 0.08903079479932785, 'ndcg@30': 0.09877375513315201, 'ndcg@50': 0.11589748412370682, 'ndcg@all': 0.38783103227615356}
Epoch 1 7.630794572551348 8.119166223626388
Epoch 2 5.741322891056886 7.372438832333214
Epoch 3 4.70718807226036 7.003115578701622
Epoch 4 4.040690806874058 6.7176112626728255
Epoch 5 3.56652854060569 6.533565847497237
Epoch 6 3.208136084484078 6.433688464917634
Epoch 7 2.9337398266931722 6.33289

Epoch 136 0.4292029172007801 5.65837456050672
Epoch 137 0.4282070572613276 5.65800696925113
Epoch 138 0.42720581059567414 5.6559469574376156
Epoch 139 0.42479992121980903 5.6543067630968595
Epoch 140 0.4231208271799032 5.653824831310072
Epoch 141 0.4224900303528323 5.66227388381958
Epoch 142 0.41994188874088534 5.658804316269724
Epoch 143 0.4174054937753064 5.663816100672672
Epoch 144 0.41673597432019416 5.66397732182553
Epoch 145 0.41485506958431667 5.662386392292223
Epoch 146 0.41262536456710414 5.656373827080977
Epoch 147 0.4106323590055544 5.669774457028038
Epoch 148 0.40890299454767104 5.660621542679636
Epoch 149 0.40782701742579364 5.664152998673289
Epoch 150 0.4057483878749156 5.659847083844636
Time1:  0.48781407438218594
Time2:  0.05162840895354748
Time3:  0.16418270207941532
Time1:  0.05432446487247944
Time2:  0.008219212293624878
Time3:  0.018908094614744186

train {'precision@10': 0.549035370349884, 'precision@30': 0.5579506754875183, 'precision@50': 0.5615457892417908, 'ndc

Epoch 268 0.2918160395664081 5.740734777952495
Epoch 269 0.29151924625474807 5.739767099681654
Epoch 270 0.29027844956743787 5.739340430811832
Epoch 271 0.2897665636581287 5.7408826476649235
Epoch 272 0.28847124632339033 5.739184931704872
Epoch 273 0.2889929445166337 5.741115319101434
Epoch 274 0.28884462219232704 5.743186298169587
Epoch 275 0.2881839871406555 5.742859714909604
Epoch 276 0.2868552479827613 5.739415871469598
Epoch 277 0.2861120071676042 5.740544243862755
Epoch 278 0.2865508019227033 5.742963088186164
Epoch 279 0.28664050551882964 5.741399413660953
Epoch 280 0.2853217271336338 5.7383442176015755
Epoch 281 0.2838943107434881 5.750457964445415
Epoch 282 0.2830490241622367 5.748042307401958
Epoch 283 0.2840631930103079 5.747386681406121
Epoch 284 0.28254524770884487 5.7449340318378646
Epoch 285 0.2829003910920773 5.747419633363423
Epoch 286 0.2825185234783686 5.75143605784366
Epoch 287 0.2806964277001152 5.744705576645701
Epoch 288 0.28042446190153647 5.749707046308015
Epoc

Epoch 401 0.23550660132664686 5.820138881081029
Epoch 402 0.23481796464027718 5.819893887168483
Epoch 403 0.2346050953655912 5.8203007296512
Epoch 404 0.23460334757266685 5.812806606292725
Epoch 405 0.23422171044767948 5.820287729564466
Epoch 406 0.23385358353455862 5.823027962132504
Epoch 407 0.2333930906845115 5.825497250807913
Epoch 408 0.23288103240972374 5.825371265411377
Epoch 409 0.2341129078328261 5.8256534275255705
Epoch 410 0.23251494242433915 5.825591012051231
Epoch 411 0.23350451912796288 5.832686850899144
Epoch 412 0.23222543281769892 5.827295629601729
Epoch 413 0.231398525729514 5.82989660062288
Epoch 414 0.23154000606801775 5.826279138263903
Epoch 415 0.23120858559482976 5.832578809637773
Epoch 416 0.23145011529239298 5.8320183753967285
Epoch 417 0.23001703794239559 5.827104869641755
Epoch 418 0.2305875200974314 5.830052175019917
Epoch 419 0.23068695563321923 5.829786928076493
Epoch 420 0.2300387060607386 5.828658329813104
Epoch 421 0.22938153862256055 5.828743231923957


...

Epoch 4420 0.07741389427965845 7.037549018859863
Epoch 4421 0.0774901384237217 7.0338822164033585
Epoch 4422 0.07713585843642552 7.031951979586952
Epoch 4423 0.07724913527742464 7.041886254360802
Epoch 4424 0.07683403441431927 7.0361193606728
Epoch 4425 0.07754857276092496 7.033593629535876
Epoch 4426 0.07757229157532865 7.036704866509688
Epoch 4427 0.0774683110110941 7.036513780292712
Epoch 4428 0.07718088883056975 7.041012186753123
Epoch 4429 0.07785510703137047 7.038277801714446
Epoch 4430 0.07746072039443846 7.043885732951917
Epoch 4431 0.07712128745359287 7.048231375844855
Epoch 4432 0.07702300980774282 7.041406004052413
Epoch 4433 0.07752352347325163 7.040489096390574
Epoch 4434 0.0770272777268761 7.04215649554604
Epoch 4435 0.07751737426073231 7.0435934568706315
Epoch 4436 0.07699117552467256 7.041855485815751
Epoch 4437 0.07779535298284732 7.042866832331607
Epoch 4438 0.07754639431572798 7.043884804374294
Epoch 4439 0.07737019209311022 7.041373177578575
Epoch 4440 0.07727654766

Epoch 4551 0.07666044712763781 7.06122360731426
Epoch 4552 0.07610992590586345 7.069509832482589
Epoch 4553 0.0767347830516553 7.063116148898476
Epoch 4554 0.07644956959792745 7.065639420559532
Epoch 4555 0.07695401024225859 7.065969893806859
Epoch 4556 0.07632868933050256 7.066195989909925
Epoch 4557 0.07618135788984466 7.0661822871158
Epoch 4558 0.07646072830198801 7.066916139502275
Epoch 4559 0.07633161279018859 7.064556272406327
Epoch 4560 0.07679096077792129 7.070150877300062
Epoch 4561 0.07689602002065782 7.069032167133532
Epoch 4562 0.07610240304156353 7.0676799824363306
Epoch 4563 0.07632450310633196 7.066681736393979
Epoch 4564 0.07643127789971424 7.068909544693796
Epoch 4565 0.07670640710153077 7.065959704549689
Epoch 4566 0.07655732843436693 7.066118265453138
Epoch 4567 0.07589931903701079 7.070708525808234
Epoch 4568 0.07661366941984633 7.067647256349263
Epoch 4569 0.07630262332178696 7.0649818119249845
Epoch 4570 0.07664957951906828 7.072997971584923
Epoch 4571 0.076329694

Epoch 4693 0.07549281466251229 7.08869941611039
Epoch 4694 0.07584924782403031 7.092229391399183
Epoch 4695 0.07536059652852733 7.100721610219855
Epoch 4696 0.07551679254798165 7.090002586967067
Epoch 4697 0.07546061586741118 7.089119610033538
Epoch 4698 0.07566381446276492 7.092567895588122
Epoch 4699 0.07601361665112233 7.090487204099956
Epoch 4700 0.0752817799299084 7.088813580964741
Time1:  0.4886180441826582
Time2:  0.04987652786076069
Time3:  0.1637501996010542
Time1:  0.05440474860370159
Time2:  0.008071228861808777
Time3:  0.018833201378583908

train {'precision@10': 0.6484020352363586, 'precision@30': 0.7053499817848206, 'precision@50': 0.7107007503509521, 'ndcg@10': 0.5307245850563049, 'ndcg@30': 0.6524545550346375, 'ndcg@50': 0.7431520819664001, 'ndcg@all': 0.7528306841850281}
valid {'precision@10': 0.4177308976650238, 'precision@30': 0.3217414617538452, 'precision@50': 0.27331921458244324, 'ndcg@10': 0.3849017322063446, 'ndcg@30': 0.3813273310661316, 'ndcg@50': 0.3960969150

Epoch 4821 0.07471378744519942 7.109301968624718
Epoch 4822 0.07450176588101694 7.105955224288137
Epoch 4823 0.07489262978758729 7.113927715703061
Epoch 4824 0.07533428865915154 7.110732530292712
Epoch 4825 0.07450341624997513 7.113054426092851
Epoch 4826 0.07505470004520919 7.107476058759187
Epoch 4827 0.07467740708798692 7.120363084893477
Epoch 4828 0.07512135503062031 7.114572299154181
Epoch 4829 0.07452628208182709 7.117935306147525
Epoch 4830 0.07452633802653753 7.111491705241956
Epoch 4831 0.07465025669301463 7.111699631339626
Epoch 4832 0.0750370801192278 7.109407851570531
Epoch 4833 0.07489248522018131 7.110749319980019
Epoch 4834 0.07459740859200382 7.117331454628392
Epoch 4835 0.07449727867081848 7.11566360373246
Epoch 4836 0.07445408369016926 7.11579694245991
Epoch 4837 0.07453008327219221 7.113644499527781
Epoch 4838 0.0746192558292757 7.116317096509431
Epoch 4839 0.0746051164572699 7.112031409614964
Epoch 4840 0.07488164793678194 7.119180051903975
Epoch 4841 0.074169893405

Epoch 4951 0.07397593048058058 7.137382457130833
Epoch 4952 0.07337395999340983 7.136935560326827
Epoch 4953 0.07397205770364282 7.139230050538716
Epoch 4954 0.07424376595612855 7.135972750814338
Epoch 4955 0.07418042133774674 7.139362259915001
Epoch 4956 0.0742234420358089 7.131974345759342
Epoch 4957 0.07431088592748196 7.13766958839015
Epoch 4958 0.07391684759430021 7.135665517104299
Epoch 4959 0.07409835968449799 7.139606902473851
Epoch 4960 0.07338012839879907 7.148129111842105
Epoch 4961 0.07464625346556045 7.140285140589664
Epoch 4962 0.07385740555517854 7.139320197858308
Epoch 4963 0.07387916602150739 7.1337527224892066
Epoch 4964 0.07391641916413057 7.137392596194618
Epoch 4965 0.07407160552098738 7.139000340511925
Epoch 4966 0.0736810290064031 7.136038479052092
Epoch 4967 0.0740282325542461 7.141219866903205
Epoch 4968 0.07381757701698102 7.1382036209106445
Epoch 4969 0.07420801841898968 7.1379829456931665
Epoch 4970 0.07367140115701665 7.140657399830065
Epoch 4971 0.07393752

In [26]:
def listNet(y_pred, y_true, eps=1e-10, padded_value_indicator=-1):
    """
    ListNet loss introduced in "Learning to Rank: From Pairwise Approach to Listwise Approach".
    :param y_pred: predictions from the model, shape [batch_size, slate_length]
    :param y_true: ground truth labels, shape [batch_size, slate_length]
    :param eps: epsilon value, used for numerical stability
    :param padded_value_indicator: an indicator of the y_true index containing a padded item, e.g. -1
    :return: loss value, a torch.Tensor
    """
    y_pred = y_pred.clone()
    y_true = y_true.clone()

    mask = y_true == padded_value_indicator
    y_pred[mask] = float('-inf')
    y_true[mask] = float('-inf')

    preds_smax = F.softmax(y_pred, dim=1)
    true_smax = F.softmax(y_true, dim=1)

    preds_smax = preds_smax + eps
    preds_log = torch.log(preds_smax)

    return torch.mean(-torch.sum(true_smax * preds_log, dim=1))

In [27]:
# setting
lr = 0.1
momentum = 0.
weight_decay = 0
nesterov = False # True

n_epoch = 5000
verbose = True
valid_epoch = 50

h_dim = 3000

model = MLPDecoder(doc_emb_dim=document_vectors.shape[1], num_words=len(word_vectors), h_dim=h_dim).to(device)
model.train()

word_vectors_tensor = torch.FloatTensor(word_vectors).to(device)

opt = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay, nesterov=nesterov)
# opt = torch.optim.Adam(model.parameters(), lr=lr)
# criterion = nn.MSELoss(reduction='mean')
# criterion = nn.MultiLabelMarginLoss(reduction='mean')
# criterion = nn.MultiLabelSoftMarginLoss(reduction='mean')

results = []
step = 0
clip_value = 1

for epoch in tqdm(range(n_epoch)):    
    train_loss_his = []
    valid_loss_his = []
    
    model.train()

    for data in train_loader:
        doc_embs, target, target_rank = data
        
        doc_embs = doc_embs.to(device)
        target = target.to(device)
        target_rank = target_rank.to(device)
        
        pred = model(doc_embs)     
        loss = listNet(pred, target)
    
        # Model backwarding
        model.zero_grad()
        loss.backward()
        
        opt.step()

        train_loss_his.append(loss.item())
        
    model.eval()
    for data in valid_loader:
        doc_embs, target, target_rank = data
        
        doc_embs = doc_embs.to(device)
        target = target.to(device)
        target_rank = target_rank.to(device)
        
        # MSE loss
        pred = model(doc_embs)     
        loss = listNet(pred, target)
    
        valid_loss_his.append(loss.item())
    
    print("Epoch", epoch, np.mean(train_loss_his), np.mean(valid_loss_his))
    
    if epoch % valid_epoch == 0:
        res = {}
        res['epoch'] = epoch
        
        train_res_ndcg = evaluate_MLPDecoder(model, train_loader)
        valid_res_ndcg = evaluate_MLPDecoder(model, valid_loader)
        
        res.update(valid_res_ndcg)
        results.append(res)
        
        if verbose:
            print()
            print('train', train_res_ndcg)
            print('valid', valid_res_ndcg)


Epoch 0 7.558826981929311 6.642893239071495
Time1:  0.48111855424940586
Time2:  0.050766244530677795
Time3:  0.1644516959786415
Time1:  0.05355365201830864
Time2:  0.008210383355617523
Time3:  0.018949346616864204

train {'precision@10': 0.19491584599018097, 'precision@30': 0.11187867075204849, 'precision@50': 0.08433237671852112, 'ndcg@10': 0.24771833419799805, 'ndcg@30': 0.21983011066913605, 'ndcg@50': 0.21720483899116516, 'ndcg@all': 0.4307364821434021}
valid {'precision@10': 0.18659630417823792, 'precision@30': 0.10979772359132767, 'precision@50': 0.08303956687450409, 'ndcg@10': 0.21357251703739166, 'ndcg@30': 0.1910678893327713, 'ndcg@50': 0.18884526193141937, 'ndcg@all': 0.40859586000442505}
Epoch 1 5.386427056719685 5.65228974191766
Epoch 2 4.122493099748042 5.1396158870897795
Epoch 3 3.3047335440652414 4.75669695201673
Epoch 4 2.7447768749549373 4.531344840401097
Epoch 5 2.3409588950419287 4.370237086948595
Epoch 6 2.0675212883809855 4.255293871227064
Epoch 7 1.8832298001350716

Epoch 135 1.2597397468243425 3.406078526848241
Epoch 136 1.2599005632930331 3.4035764367956864
Epoch 137 1.2579292058944702 3.406371493088572
Epoch 138 1.258521216654638 3.3975021337207996
Epoch 139 1.2588074437358923 3.4017832028238395
Epoch 140 1.2572579042256227 3.404090680574116
Epoch 141 1.2588042048682944 3.3992137407001697
Epoch 142 1.2595475047652485 3.402209294469733
Epoch 143 1.257382897954238 3.3968052738591243
Epoch 144 1.2562592008657623 3.388517166438856
Epoch 145 1.2570915612560964 3.4015126102849056
Epoch 146 1.2568077214977198 3.39712875767758
Epoch 147 1.2569824631451165 3.390445960195441
Epoch 148 1.256877797388891 3.398538087543688
Epoch 149 1.2571047331854615 3.3956476010774312
Epoch 150 1.2566403635064063 3.3917504988218607
Time1:  0.47929034382104874
Time2:  0.05126351863145828
Time3:  0.1642293818295002
Time1:  0.05348225310444832
Time2:  0.008181614801287651
Time3:  0.018965011462569237

train {'precision@10': 0.5086261034011841, 'precision@30': 0.2592564523220

Epoch 268 1.2403727449171724 3.3510616327586926
Epoch 269 1.2396841028280425 3.3480429900319955
Epoch 270 1.2399257832800437 3.3476637037176835
Epoch 271 1.2394402009701868 3.3426146256296256
Epoch 272 1.2402539016210545 3.348825504905299
Epoch 273 1.2388543796818159 3.353108569195396
Epoch 274 1.2400201036219012 3.3402598908073022
Epoch 275 1.2410284183178728 3.346604447615774
Epoch 276 1.2411729707355388 3.350219312467073
Epoch 277 1.2371617918126068 3.3472148619200053
Epoch 278 1.2391816178957622 3.3592327519467005
Epoch 279 1.2391265205472533 3.34495600901152
Epoch 280 1.2390639088307207 3.3446347964437386
Epoch 281 1.2383231388198004 3.343806643235056
Epoch 282 1.2395718156942848 3.3396058459030953
Epoch 283 1.2391244985206782 3.343995257427818
Epoch 284 1.238725067230693 3.342506270659597
Epoch 285 1.2382654662717854 3.348313281410619
Epoch 286 1.2413311304404722 3.3434331668050667
Epoch 287 1.2381334757944296 3.3505653080187345
Epoch 288 1.23868955924497 3.347330883929604
Epoch 

Epoch 401 1.2319392032093472 3.340606451034546
Epoch 402 1.2317541792378788 3.333127762141981
Epoch 403 1.230491468432354 3.338856421018902
Epoch 404 1.2304626197842827 3.3317849008660567
Epoch 405 1.232363504275941 3.3356052448875024
Epoch 406 1.2315258958883453 3.335697186620612
Epoch 407 1.2306940830938997 3.339101276899639
Epoch 408 1.2328254270274737 3.329742619865819
Epoch 409 1.232120604891526 3.3404737146277177
Epoch 410 1.231837414858634 3.3304218994943717
Epoch 411 1.2308993987869798 3.3323654375578227
Epoch 412 1.2301074364031965 3.334660028156481
Epoch 413 1.2300295157042163 3.3296350428932593
Epoch 414 1.2319474680381908 3.334347737462897
Epoch 415 1.2293350661707203 3.3356498668068335
Epoch 416 1.231697606761553 3.327612048701236
Epoch 417 1.2311688675517924 3.329871993315847
Epoch 418 1.2305808520456503 3.3337048856835616
Epoch 419 1.2297281120952808 3.3285773804313257
Epoch 420 1.230550779933818 3.3309754321449683
Epoch 421 1.2288366580567165 3.3297368476265357
Epoch 42

Epoch 547 1.2256234831977308 3.3380564513959383
Epoch 548 1.227158390639121 3.3394918065322075
Epoch 549 1.2255926320427342 3.3335102231878984
Epoch 550 1.2264449690517627 3.3332309095483077
Time1:  0.4793894086033106
Time2:  0.05086854100227356
Time3:  0.16421302035450935
Time1:  0.05353509820997715
Time2:  0.008177397772669792
Time3:  0.01894674450159073

train {'precision@10': 0.5663167834281921, 'precision@30': 0.2923806309700012, 'precision@50': 0.20947866141796112, 'ndcg@10': 0.7500427961349487, 'ndcg@30': 0.6242126822471619, 'ndcg@50': 0.6043657064437866, 'ndcg@all': 0.7580590844154358}
valid {'precision@10': 0.467387855052948, 'precision@30': 0.2571679949760437, 'precision@50': 0.18930870294570923, 'ndcg@10': 0.5848190188407898, 'ndcg@30': 0.5023055672645569, 'ndcg@50': 0.49015316367149353, 'ndcg@all': 0.6654290556907654}
Epoch 551 1.2242247525014376 3.338511667753521
Epoch 552 1.224712240068536 3.3363934441616663
Epoch 553 1.2255599129269694 3.3311534555334794
Epoch 554 1.2261

...

Epoch 4324 1.1902811924616497 3.717664969594855
Epoch 4325 1.189992007804893 3.7245326167658757
Epoch 4326 1.192484137956162 3.718633162347894
Epoch 4327 1.1916824924318414 3.7131298843183016
Epoch 4328 1.1921954639473853 3.7124556114799097
Epoch 4329 1.1918721997249893 3.7164385444239567
Epoch 4330 1.1919129968386644 3.7136423713282536
Epoch 4331 1.1913063902603953 3.718430468910619
Epoch 4332 1.1921423921111034 3.7170152538701107
Epoch 4333 1.1928337784538492 3.7141535533101937
Epoch 4334 1.190712336908307 3.715080687874242
Epoch 4335 1.1903164616802282 3.7170440146797583
Epoch 4336 1.1918157269383034 3.7129639324389005
Epoch 4337 1.1911606419156169 3.721477596383346
Epoch 4338 1.1904674242114464 3.71507423802426
Epoch 4339 1.190175391777217 3.718521682839645
Epoch 4340 1.1914261586484853 3.714700749045924
Epoch 4341 1.1924030599538347 3.7183845670599687
Epoch 4342 1.1888802818387572 3.716432860023097
Epoch 4343 1.1901181131078487 3.718793605503283
Epoch 4344 1.191279693305144 3.7137

Epoch 4454 1.1897095072339152 3.7263473209581877
Epoch 4455 1.1907009660151966 3.7259935328834937
Epoch 4456 1.1920762811487877 3.7280005279340243
Epoch 4457 1.1896402089916476 3.723363060700266
Epoch 4458 1.19157490256237 3.72905579366182
Epoch 4459 1.191259024784579 3.7277876954329643
Epoch 4460 1.1903925586862174 3.724735699201885
Epoch 4461 1.1896651917033725 3.731282999641017
Epoch 4462 1.1899849929307635 3.7213162999404106
Epoch 4463 1.1897990804666665 3.7340313384407446
Epoch 4464 1.1903305893753007 3.7259052050741097
Epoch 4465 1.1901094749657035 3.7244349529868677
Epoch 4466 1.1903130614269546 3.7312541258962533
Epoch 4467 1.1901471144274662 3.7271989521227384
Epoch 4468 1.1909509761291637 3.726009833185296
Epoch 4469 1.1910028970032407 3.7241711741999577
Epoch 4470 1.1913873372022172 3.7236947511371814
Epoch 4471 1.1903377231101544 3.724169505269904
Epoch 4472 1.19044104758759 3.727147491354691
Epoch 4473 1.1913169479509542 3.7220175140782406
Epoch 4474 1.191837723840747 3.72

Epoch 4597 1.1913482208698116 3.730882067429392
Epoch 4598 1.1897837599815682 3.727907268624557
Epoch 4599 1.1908460203667133 3.7391460820248255
Epoch 4600 1.189570987433718 3.7268323270898116
Time1:  0.48012007586658
Time2:  0.04951302148401737
Time3:  0.16425329819321632
Time1:  0.05348131246864796
Time2:  0.008081275969743729
Time3:  0.019031476229429245

train {'precision@10': 0.6353134512901306, 'precision@30': 0.3506421148777008, 'precision@50': 0.25204360485076904, 'ndcg@10': 0.7927646636962891, 'ndcg@30': 0.6751971244812012, 'ndcg@50': 0.6560013890266418, 'ndcg@all': 0.7938013076782227}
valid {'precision@10': 0.4852770268917084, 'precision@30': 0.2764819860458374, 'precision@50': 0.20608969032764435, 'ndcg@10': 0.5765466690063477, 'ndcg@30': 0.5036401152610779, 'ndcg@50': 0.49409908056259155, 'ndcg@all': 0.6680598258972168}
Epoch 4601 1.1896837827754996 3.732344075253135
Epoch 4602 1.1893633942157902 3.729081028386166
Epoch 4603 1.1901702636863754 3.7388489497335335
Epoch 4604 

Epoch 4727 1.1892588867081537 3.7468808575680383
Epoch 4728 1.1904896062019972 3.744225288692274
Epoch 4729 1.189841447866451 3.7370326268045524
Epoch 4730 1.1903995739089117 3.7434855134863603
Epoch 4731 1.1915097888450177 3.739301505841707
Epoch 4732 1.1893500866945723 3.7451886126869605
Epoch 4733 1.1910221691717182 3.7417986894908704
Epoch 4734 1.1905440953042772 3.7455725293410453
Epoch 4735 1.190063734849294 3.741906580172087
Epoch 4736 1.189807280810953 3.7416234267385384
Epoch 4737 1.1899431926465174 3.737865836996781
Epoch 4738 1.189521410660437 3.7467691270928634
Epoch 4739 1.1896859027488886 3.743713742808292
Epoch 4740 1.1915455823056182 3.7458613044337223
Epoch 4741 1.1917886078706261 3.743445772873728
Epoch 4742 1.1898383777043973 3.744497675644724
Epoch 4743 1.189900403134307 3.7456717114699516
Epoch 4744 1.1903628473393402 3.735151454022056
Epoch 4745 1.1900857819451227 3.7497260319559196
Epoch 4746 1.192620342935038 3.7460539717423287
Epoch 4747 1.1895781755447388 3.74

Epoch 4857 1.1910690675702011 3.744597309514096
Epoch 4858 1.1888161984800596 3.7496253942188464
Epoch 4859 1.1893282797601488 3.7490815237948767
Epoch 4860 1.1891282204298945 3.744137977298937
Epoch 4861 1.19089509834323 3.7531441387377287
Epoch 4862 1.1901542220199317 3.7535357224313834
Epoch 4863 1.1908886307164241 3.7562148069080554
Epoch 4864 1.1901121132555064 3.7559164950722144
Epoch 4865 1.188481344465624 3.7525775683553597
Epoch 4866 1.1907926627069887 3.7535735305986906
Epoch 4867 1.1898452217815911 3.7529687254052413
Epoch 4868 1.1888747734633105 3.7483897083684017
Epoch 4869 1.189007830550099 3.7516904002741764
Epoch 4870 1.1901632997027614 3.7468401632810893
Epoch 4871 1.191444562889679 3.745100824456466
Epoch 4872 1.188799940703208 3.75021263172752
Epoch 4873 1.189607061837849 3.74874798875106
Epoch 4874 1.1897988511107818 3.7530726759057296
Epoch 4875 1.1896258017473054 3.7551655518381217
Epoch 4876 1.190208353494343 3.751370404895983
Epoch 4877 1.1892231591263709 3.7529

In [28]:
def deterministic_neural_sort(s, tau, mask):
    """
    Deterministic neural sort.
    Code taken from "Stochastic Optimization of Sorting Networks via Continuous Relaxations", ICLR 2019.
    Minor modifications applied to the original code (masking).
    :param s: values to sort, shape [batch_size, slate_length]
    :param tau: temperature for the final softmax function
    :param mask: mask indicating padded elements
    :return: approximate permutation matrices of shape [batch_size, slate_length, slate_length]
    """
    dev = get_torch_device()

    n = s.size()[1]
    one = torch.ones((n, 1), dtype=torch.float32, device=dev)
    s = s.masked_fill(mask[:, :, None], -1e8)
    A_s = torch.abs(s - s.permute(0, 2, 1))
    A_s = A_s.masked_fill(mask[:, :, None] | mask[:, None, :], 0.0)

    B = torch.matmul(A_s, torch.matmul(one, torch.transpose(one, 0, 1)))

    temp = [n - m + 1 - 2 * (torch.arange(n - m, device=dev) + 1) for m in mask.squeeze(-1).sum(dim=1)]
    temp = [t.type(torch.float32) for t in temp]
    temp = [torch.cat((t, torch.zeros(n - len(t), device=dev))) for t in temp]
    scaling = torch.stack(temp).type(torch.float32).to(dev)  # type: ignore

    s = s.masked_fill(mask[:, :, None], 0.0)
    C = torch.matmul(s, scaling.unsqueeze(-2))

    P_max = (C - B).permute(0, 2, 1)
    P_max = P_max.masked_fill(mask[:, :, None] | mask[:, None, :], -np.inf)
    P_max = P_max.masked_fill(mask[:, :, None] & mask[:, None, :], 1.0)
    sm = torch.nn.Softmax(-1)
    P_hat = sm(P_max / tau)
    return P_hat

def sinkhorn_scaling(mat, mask=None, tol=1e-6, max_iter=50):
    """
    Sinkhorn scaling procedure.
    :param mat: a tensor of square matrices of shape N x M x M, where N is batch size
    :param mask: a tensor of masks of shape N x M
    :param tol: Sinkhorn scaling tolerance
    :param max_iter: maximum number of iterations of the Sinkhorn scaling
    :return: a tensor of (approximately) doubly stochastic matrices
    """
    if mask is not None:
        mat = mat.masked_fill(mask[:, None, :] | mask[:, :, None], 0.0)
        mat = mat.masked_fill(mask[:, None, :] & mask[:, :, None], 1.0)

    for _ in range(max_iter):
        mat = mat / mat.sum(dim=1, keepdim=True).clamp(min=DEFAULT_EPS)
        mat = mat / mat.sum(dim=2, keepdim=True).clamp(min=DEFAULT_EPS)

        if torch.max(torch.abs(mat.sum(dim=2) - 1.)) < tol and torch.max(torch.abs(mat.sum(dim=1) - 1.)) < tol:
            break

    if mask is not None:
        mat = mat.masked_fill(mask[:, None, :] | mask[:, :, None], 0.0)

    return mat

def stochastic_neural_sort(s, n_samples, tau, mask, beta=1.0, log_scores=True, eps=1e-10):
    """
    Stochastic neural sort. Please note that memory complexity grows by factor n_samples.
    Code taken from "Stochastic Optimization of Sorting Networks via Continuous Relaxations", ICLR 2019.
    Minor modifications applied to the original code (masking).
    :param s: values to sort, shape [batch_size, slate_length]
    :param n_samples: number of samples (approximations) for each permutation matrix
    :param tau: temperature for the final softmax function
    :param mask: mask indicating padded elements
    :param beta: scale parameter for the Gumbel distribution
    :param log_scores: whether to apply the logarithm function to scores prior to Gumbel perturbation
    :param eps: epsilon for the logarithm function
    :return: approximate permutation matrices of shape [n_samples, batch_size, slate_length, slate_length]
    """
    dev = get_torch_device()

    batch_size = s.size()[0]
    n = s.size()[1]
    s_positive = s + torch.abs(s.min())
    samples = beta * sample_gumbel([n_samples, batch_size, n, 1], device=dev)
    if log_scores:
        s_positive = torch.log(s_positive + eps)

    s_perturb = (s_positive + samples).view(n_samples * batch_size, n, 1)
    mask_repeated = mask.repeat_interleave(n_samples, dim=0)

    P_hat = deterministic_neural_sort(s_perturb, tau, mask_repeated)
    P_hat = P_hat.view(n_samples, batch_size, n, n)
    return P_hat

def dcg(y_pred, y_true, ats=None, gain_function=lambda x: torch.pow(2, x) - 1, padding_indicator=PADDED_Y_VALUE):
    """
    Discounted Cumulative Gain at k.
    Compute DCG at ranks given by ats or at the maximum rank if ats is None.
    :param y_pred: predictions from the model, shape [batch_size, slate_length]
    :param y_true: ground truth labels, shape [batch_size, slate_length]
    :param ats: optional list of ranks for DCG evaluation, if None, maximum rank is used
    :param gain_function: callable, gain function for the ground truth labels, e.g. torch.pow(2, x) - 1
    :param padding_indicator: an indicator of the y_true index containing a padded item, e.g. -1
    :return: DCG values for each slate and evaluation position, shape [batch_size, len(ats)]
    """
    y_true = y_true.clone()
    y_pred = y_pred.clone()

    actual_length = y_true.shape[1]

    if ats is None:
        ats = [actual_length]
    ats = [min(at, actual_length) for at in ats]

    true_sorted_by_preds = __apply_mask_and_get_true_sorted_by_preds(y_pred, y_true, padding_indicator)

    discounts = (torch.tensor(1) / torch.log2(torch.arange(true_sorted_by_preds.shape[1], dtype=torch.float) + 2.0)).to(
        device=true_sorted_by_preds.device)

    gains = gain_function(true_sorted_by_preds)

    discounted_gains = (gains * discounts)[:, :np.max(ats)]

    cum_dcg = torch.cumsum(discounted_gains, dim=1)

    ats_tensor = torch.tensor(ats, dtype=torch.long) - torch.tensor(1)

    dcg = cum_dcg[:, ats_tensor]

    return dcg

def get_torch_device():
    """
    Getter for an available pyTorch device.
    :return: CUDA-capable GPU if available, CPU otherwise
    """
    return "cpu"
    return torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")


def neuralNDCG(y_pred, y_true, padded_value_indicator=PADDED_Y_VALUE, temperature=1., powered_relevancies=True, k=None,
               stochastic=False, n_samples=32, beta=0.1, log_scores=True):
    """
    NeuralNDCG loss introduced in "NeuralNDCG: Direct Optimisation of a Ranking Metric via Differentiable
    Relaxation of Sorting" - https://arxiv.org/abs/2102.07831. Based on the NeuralSort algorithm.
    :param y_pred: predictions from the model, shape [batch_size, slate_length]
    :param y_true: ground truth labels, shape [batch_size, slate_length]
    :param padded_value_indicator: an indicator of the y_true index containing a padded item, e.g. -1
    :param temperature: temperature for the NeuralSort algorithm
    :param powered_relevancies: whether to apply 2^x - 1 gain function, x otherwise
    :param k: rank at which the loss is truncated
    :param stochastic: whether to calculate the stochastic variant
    :param n_samples: how many stochastic samples are taken, used if stochastic == True
    :param beta: beta parameter for NeuralSort algorithm, used if stochastic == True
    :param log_scores: log_scores parameter for NeuralSort algorithm, used if stochastic == True
    :return: loss value, a torch.Tensor
    """
    dev = get_torch_device()

    if k is None:
        k = y_true.shape[1]

    mask = (y_true == padded_value_indicator)
    # Choose the deterministic/stochastic variant
    if stochastic:
        P_hat = stochastic_neural_sort(y_pred.unsqueeze(-1), n_samples=n_samples, tau=temperature, mask=mask,
                                       beta=beta, log_scores=log_scores)
    else:
        P_hat = deterministic_neural_sort(y_pred.unsqueeze(-1), tau=temperature, mask=mask).unsqueeze(0)

    # Perform sinkhorn scaling to obtain doubly stochastic permutation matrices
    P_hat = sinkhorn_scaling(P_hat.view(P_hat.shape[0] * P_hat.shape[1], P_hat.shape[2], P_hat.shape[3]),
                             mask.repeat_interleave(P_hat.shape[0], dim=0), tol=1e-6, max_iter=50)
    P_hat = P_hat.view(int(P_hat.shape[0] / y_pred.shape[0]), y_pred.shape[0], P_hat.shape[1], P_hat.shape[2])

    # Mask P_hat and apply to true labels, ie approximately sort them
    P_hat = P_hat.masked_fill(mask[None, :, :, None] | mask[None, :, None, :], 0.)
    y_true_masked = y_true.masked_fill(mask, 0.).unsqueeze(-1).unsqueeze(0)
    if powered_relevancies:
        y_true_masked = torch.pow(2., y_true_masked) - 1.

    ground_truth = torch.matmul(P_hat, y_true_masked).squeeze(-1)
    discounts = (torch.tensor(1.) / torch.log2(torch.arange(y_true.shape[-1], dtype=torch.float) + 2.)).to(dev)
    discounted_gains = ground_truth * discounts

    if powered_relevancies:
        idcg = dcg(y_true, y_true, ats=[k]).permute(1, 0)
    else:
        idcg = dcg(y_true, y_true, ats=[k], gain_function=lambda x: x).permute(1, 0)

    discounted_gains = discounted_gains[:, :, :k]
    ndcg = discounted_gains.sum(dim=-1) / (idcg + DEFAULT_EPS)
    idcg_mask = idcg == 0.
    ndcg = ndcg.masked_fill(idcg_mask.repeat(ndcg.shape[0], 1), 0.)

    assert (ndcg < 0.).sum() >= 0, "every ndcg should be non-negative"
    if idcg_mask.all():
        return torch.tensor(0.)

    mean_ndcg = ndcg.sum() / ((~idcg_mask).sum() * ndcg.shape[0])  # type: ignore
    return -1. * mean_ndcg  # -1 cause we want to maximize NDCG


def neuralNDCG_transposed(y_pred, y_true, padded_value_indicator=PADDED_Y_VALUE, temperature=1.,
                          powered_relevancies=True, k=None, stochastic=False, n_samples=32, beta=0.1, log_scores=True,
                          max_iter=50, tol=1e-6):
    """
    NeuralNDCG Transposed loss introduced in "NeuralNDCG: Direct Optimisation of a Ranking Metric via Differentiable
    Relaxation of Sorting" - https://arxiv.org/abs/2102.07831. Based on the NeuralSort algorithm.
    :param y_pred: predictions from the model, shape [batch_size, slate_length]
    :param y_true: ground truth labels, shape [batch_size, slate_length]
    :param padded_value_indicator: an indicator of the y_true index containing a padded item, e.g. -1
    :param temperature: temperature for the NeuralSort algorithm
    :param powered_relevancies: whether to apply 2^x - 1 gain function, x otherwise
    :param k: rank at which the loss is truncated
    :param stochastic: whether to calculate the stochastic variant
    :param n_samples: how many stochastic samples are taken, used if stochastic == True
    :param beta: beta parameter for NeuralSort algorithm, used if stochastic == True
    :param log_scores: log_scores parameter for NeuralSort algorithm, used if stochastic == True
    :param max_iter: maximum iteration count for Sinkhorn scaling
    :param tol: tolerance for Sinkhorn scaling
    :return: loss value, a torch.Tensor
    """
    dev = get_torch_device()

    if k is None:
        k = y_true.shape[1]

    mask = (y_true == padded_value_indicator)

    if stochastic:
        P_hat = stochastic_neural_sort(y_pred.unsqueeze(-1), n_samples=n_samples, tau=temperature, mask=mask,
                                       beta=beta, log_scores=log_scores)
    else:
        P_hat = deterministic_neural_sort(y_pred.unsqueeze(-1), tau=temperature, mask=mask).unsqueeze(0)

    # Perform sinkhorn scaling to obtain doubly stochastic permutation matrices
    P_hat_masked = sinkhorn_scaling(P_hat.view(P_hat.shape[0] * y_pred.shape[0], y_pred.shape[1], y_pred.shape[1]),
                                    mask.repeat_interleave(P_hat.shape[0], dim=0), tol=tol, max_iter=max_iter)
    P_hat_masked = P_hat_masked.view(P_hat.shape[0], y_pred.shape[0], y_pred.shape[1], y_pred.shape[1])
    discounts = (torch.tensor(1) / torch.log2(torch.arange(y_true.shape[-1], dtype=torch.float) + 2.)).to(dev)

    # This takes care of the @k metric truncation - if something is @>k, it is useless and gets 0.0 discount
    discounts[k:] = 0.
    discounts = discounts[None, None, :, None]

    # Here the discounts become expected discounts
    discounts = torch.matmul(P_hat_masked.permute(0, 1, 3, 2), discounts).squeeze(-1)
    if powered_relevancies:
        gains = torch.pow(2., y_true) - 1
        discounted_gains = gains.unsqueeze(0) * discounts
        idcg = dcg(y_true, y_true, ats=[k]).squeeze()
    else:
        gains = y_true
        discounted_gains = gains.unsqueeze(0) * discounts
        idcg = dcg(y_true, y_true, ats=[k]).squeeze()

    ndcg = discounted_gains.sum(dim=2) / (idcg + DEFAULT_EPS)
    idcg_mask = idcg == 0.
    ndcg = ndcg.masked_fill(idcg_mask, 0.)

    assert (ndcg < 0.).sum() >= 0, "every ndcg should be non-negative"
    if idcg_mask.all():
        return torch.tensor(0.)

    mean_ndcg = ndcg.sum() / ((~idcg_mask).sum() * ndcg.shape[0])  # type: ignore
    return -1. * mean_ndcg  # -1 cause we want to maximize NDCG

NameError: name 'PADDED_Y_VALUE' is not defined

In [ ]:
def lambdaLoss(y_pred, y_true, eps=DEFAULT_EPS, padded_value_indicator=PADDED_Y_VALUE, weighing_scheme=None, k=None, sigma=1., mu=10.,
               reduction="sum", reduction_log="binary"):
    """
    LambdaLoss framework for LTR losses implementations, introduced in "The LambdaLoss Framework for Ranking Metric Optimization".
    Contains implementations of different weighing schemes corresponding to e.g. LambdaRank or RankNet.
    :param y_pred: predictions from the model, shape [batch_size, slate_length]
    :param y_true: ground truth labels, shape [batch_size, slate_length]
    :param eps: epsilon value, used for numerical stability
    :param padded_value_indicator: an indicator of the y_true index containing a padded item, e.g. -1
    :param weighing_scheme: a string corresponding to a name of one of the weighing schemes
    :param k: rank at which the loss is truncated
    :param sigma: score difference weight used in the sigmoid function
    :param mu: optional weight used in NDCGLoss2++ weighing scheme
    :param reduction: losses reduction method, could be either a sum or a mean
    :param reduction_log: logarithm variant used prior to masking and loss reduction, either binary or natural
    :return: loss value, a torch.Tensor
    """
    device = y_pred.device
    y_pred = y_pred.clone()
    y_true = y_true.clone()

    padded_mask = y_true == padded_value_indicator
    y_pred[padded_mask] = float("-inf")
    y_true[padded_mask] = float("-inf")

    # Here we sort the true and predicted relevancy scores.
    y_pred_sorted, indices_pred = y_pred.sort(descending=True, dim=-1)
    y_true_sorted, _ = y_true.sort(descending=True, dim=-1)

    # After sorting, we can mask out the pairs of indices (i, j) containing index of a padded element.
    true_sorted_by_preds = torch.gather(y_true, dim=1, index=indices_pred)
    true_diffs = true_sorted_by_preds[:, :, None] - true_sorted_by_preds[:, None, :]
    print(true_sorted_by_preds.shape)
    print(true_diffs.shape)
    padded_pairs_mask = torch.isfinite(true_diffs)

    if weighing_scheme != "ndcgLoss1_scheme":
        padded_pairs_mask = padded_pairs_mask & (true_diffs > 0)

    ndcg_at_k_mask = torch.zeros((y_pred.shape[1], y_pred.shape[1]), dtype=torch.bool, device=device)
    ndcg_at_k_mask[:k, :k] = 1

    # Here we clamp the -infs to get correct gains and ideal DCGs (maxDCGs)
    true_sorted_by_preds.clamp_(min=0.)
    y_true_sorted.clamp_(min=0.)

    # Here we find the gains, discounts and ideal DCGs per slate.
    pos_idxs = torch.arange(1, y_pred.shape[1] + 1).to(device)
    D = torch.log2(1. + pos_idxs.float())[None, :]
    maxDCGs = torch.sum(((torch.pow(2, y_true_sorted) - 1) / D)[:, :k], dim=-1).clamp(min=eps)
    G = (torch.pow(2, true_sorted_by_preds) - 1) / maxDCGs[:, None]

    # Here we apply appropriate weighing scheme - ndcgLoss1, ndcgLoss2, ndcgLoss2++ or no weights (=1.0)
    if weighing_scheme is None:
        weights = 1.
    else:
        weights = globals()[weighing_scheme](G, D, mu, true_sorted_by_preds)  # type: ignore

    # We are clamping the array entries to maintain correct backprop (log(0) and division by 0)
    scores_diffs = (y_pred_sorted[:, :, None] - y_pred_sorted[:, None, :]).clamp(min=-1e8, max=1e8)
    scores_diffs.masked_fill(torch.isnan(scores_diffs), 0.)
    weighted_probas = (torch.sigmoid(sigma * scores_diffs).clamp(min=eps) ** weights).clamp(min=eps)
    if reduction_log == "natural":
        losses = torch.log(weighted_probas)
    elif reduction_log == "binary":
        losses = torch.log2(weighted_probas)
    else:
        raise ValueError("Reduction logarithm base can be either natural or binary")

    if reduction == "sum":
        loss = -torch.sum(losses[padded_pairs_mask & ndcg_at_k_mask])
    elif reduction == "mean":
        loss = -torch.mean(losses[padded_pairs_mask & ndcg_at_k_mask])
    else:
        raise ValueError("Reduction method can be either sum or mean")

    return loss


def ndcgLoss1_scheme(G, D, *args):
    return (G / D)[:, :, None]


def ndcgLoss2_scheme(G, D, *args):
    pos_idxs = torch.arange(1, G.shape[1] + 1, device=G.device)
    delta_idxs = torch.abs(pos_idxs[:, None] - pos_idxs[None, :])
    deltas = torch.abs(torch.pow(torch.abs(D[0, delta_idxs - 1]), -1.) - torch.pow(torch.abs(D[0, delta_idxs]), -1.))
    deltas.diagonal().zero_()

    return deltas[None, :, :] * torch.abs(G[:, :, None] - G[:, None, :])


def lambdaRank_scheme(G, D, *args):
    return torch.abs(torch.pow(D[:, :, None], -1.) - torch.pow(D[:, None, :], -1.)) * torch.abs(G[:, :, None] - G[:, None, :])


def ndcgLoss2PP_scheme(G, D, *args):
    return args[0] * ndcgLoss2_scheme(G, D) + lambdaRank_scheme(G, D)


def rankNet_scheme(G, D, *args):
    return 1.


def rankNetWeightedByGTDiff_scheme(G, D, *args):
    return torch.abs(args[1][:, :, None] - args[1][:, None, :])


def rankNetWeightedByGTDiffPowed_scheme(G, D, *args):
    return torch.abs(torch.pow(args[1][:, :, None], 2) - torch.pow(args[1][:, None, :], 2))